# Dynamical ETH

$
\require{physics}
\def\bm{\boldsymbol}
\def\indx{\sigma_1, \dots, \sigma_L}
\def\ind{\tau_1, \dots, \tau_L}
$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from ph121c_lxvm import models, data, tests
from ph121c_lxvm.fortran import evolve
evolve.get_threads()

The model we will study in this notebook is:
$$
    H = \sum_{j=1}^L \sigma_j^z \sigma_{j+1}^z
    - h^x \sum_{j=1}^L \sigma_j^x
    - h^z \sum_{j=1}^L \sigma_j^z 
    .
$$
We are interested in the fact that $h^z \neq 0$ makes this
TFIM Hamiltonian non-integrable, and creates eigenstate thermalization.
We set the following generic parameter values:

In [ ]:
hx, hz = (-1.05, 0.5)

## Time evolution of an initial state

Our initial state will be
$$
    \ket{\psi (t=0)} = \ket{\xi}_1 \otimes \cdots \otimes \ket{\xi}_L
    ,
$$
with $\ket{\xi} = \frac{1}{2} \left( \ket{\uparrow} - \sqrt{3} \ket{\downarrow}\right)$.

In [ ]:
xi = np.array([-np.sqrt(3), 1]) * 0.5

We are interested in time-evolving this state, which we can do in the energy
eigenbasis, where:
$$
    \ket{\psi} 
        = \sum_{\indx=0}^{1} a_{\indx} \ket{\indx} 
        = \sum_{n=1}^{2^L} c_n \ket{n}
    ,
$$
where $c_n = \sum_{\indx} a_n \braket{n}{\indx}$.
The time evolution of an operator in the energy basis, where
$H \ket{n} = \epsilon_n \ket{n}$, can be described by:
$$
    \ev{O}{\psi(t)} 
        = \sum_{n, m} c_m^* c_n e^{-i(\epsilon_n - \epsilon_m)t} \mel{m}{O}{n}
    .
$$

Shu Fay told me it took hours to run code to do this, so I decided I would
think about how to cast the problem efficiently.
We could represent the sum above as the sum of the entries of a matrix in the
energy basis, where initially each element is $c_m^* c_n \mel{m}{O}{n}$ and
we update this at each time step of duration $\Delta t$ by multiplying with
$e^{-i(\epsilon_n - \epsilon_m)\Delta t}$. We would find that each time step
incurs $2^{2L}$ multiplications followed by summing $2^{2L}$ elements together.
This is a fine way of doing the problem.

I wanted to use the fact we know the Pauli operators very well in the computational
basis. In fact, we know that they have precisely $2^L$ nonzero elements, and we
can calculate these elements with bitwise operations. We can use resolution of
the identity to write:
\begin{align}
    \ev{O}{\psi(t)} 
        &= \sum_{\indx} \sum_{\indxb} \sum_{n, m}
        c_m^* c_n e^{-i(\epsilon_n - \epsilon_m)t}
        \braket{m}{\indx}\mel{\indx}{O}{\ind}\braket{\ind}{n}
    \\
        &= \sum_{\indx} \sum_{\ind} \mel{\indx}{O}{\ind}
        \left( \sum_{n} c_n e^{-i\epsilon_n t} \braket{\ind}{n} \right)
        \left( \sum_{m} c_m e^{-i\epsilon_m t} \braket{\indx}{m} \right)^*
    .
\end{align}
For $O = \sigma_k^\mu$ where $k \in \\{1, \dots, L \\}$ and $\mu \in {x, y, z}$,
we have binary formulas acting on site $k, k-1, k+1$ that give us the matrix
elements $\mel{\indx}{O}{\ind}$. In fact, we know $\ind$ to be a function of
$\indx$, so we will write $\tau(\indx) = \ind$.
Therefore we have succesfully eliminated one of the indices:
\begin{align}
    \ev{O}{\psi(t)} 
        &= \sum_{\indx} \mel{\indx}{O}{\tau(\indx)}
        \left( \sum_{n} c_n e^{-i\epsilon_n t} \braket{\tau(\indx)}{n} \right)
        \left( \sum_{m} c_m e^{-i\epsilon_m t} \braket{\indx}{m} \right)^*
    .
\end{align}
This means that when $\tau(\indx) = \indx$ which is true for diagonal operators
such as $\sigma_k^z$, we can also limit the complexity of the problem to $2^{2L}$
sums and multiplications, without ever having to change basis.
Note that in this case, $\braket{\indx}{m}$ is just a matrix element of one of
the eigenvectors returned by the diagonalization routine.

I wrote code to do this time evolution in Fortran, and basically it does a loop
over the physical index and the inner indices are just sums over vectors.
I used OpenMP directives to speed the loop over the $\indx$ index in my Fortran
code by about 25% compared to the serial version.

In short, time evolution on a temporal grid is a computationally intensive task.
I expect this to take a long time to run, so I will make sure to save the results.
After all, it did take me 4 tries before `np.linalg.eigh` didn't get itself
killed due to requesting too much memory at $L=14$. When it did work, then
$L=14$ took about 10 minutes to diagonalize (`numpy` says it uses the `syevd`
routine from LAPACK, but back in assignment 1, I used the generic `syev` routine
from MKL LAPACK and $L=14$ only took 2 minutes there).
All of the diagonalization runtime information lives in the metadata of my HDF5
archive, but I'm too slow to get it out. All I know is that my appendix to
assignment 2 demonstrates the code to talk with the archive.

In [ ]:
%%time

dt = 0.05
Nstp = 1000
bc = 'c'

save = '../../data/wobbles.pick'

try:
    df = pd.read_pickle(save)
except FileNotFoundError:

    wobbles = {
        's_0^?' : [],
        'vals' : [],
        'L' : [],
    }

    for L in [8, 10, 12, 14]:

        job = dict(
            oper=models.tfim_z.H_dense,
            oper_params={
                'L' : L,
                'h' : hx,
                'hz': hz,
                'bc': bc,
            },
            solver=np.linalg.eigh,
            solver_params={},
        )
        evals, evecs = data.jobs.obtain(**job)

        psi = 1
        for i in range(L):
            psi = np.kron(xi, psi)
        coef = evecs.T @ psi
        for which in ['x', 'y', 'z']:
            cevecs = (coef * evecs).T.astype('complex')
            tevals = np.exp(-1j*dt*evals)
            wobbles['L'].append(L)
            wobbles['s_0^?'].append(which)
            wobbles['vals'].append(
                evolve.time_ev(l=L, k=0, nstp=Nstp, which=which, cevecs=cevecs, tevals=tevals)
            )

    df = pd.DataFrame(wobbles)
    df.to_pickle(save)

In this case, I ran the code in three separate chunks, so this runtime was for
$L=14$ and `which = 'y'`.
I think I've been here for about 30 minutes, so let's just say that's the time
it took to generate all 12 time evolutions (The diagonalization was done previously).
Of that time, $L=12$ took about 30-40 seconds each, so every $L=14$ evolution
took about 10 minutes.

In [ ]:
plots = []
for i, val in enumerate(df.vals):
    fig, ax = plt.subplots()
    ax.plot(np.arange(Nstp)*dt, val)
    ax.set_title(f"$\sigma_0^{df['s_0^?'][i]}, L={df.L[i]}$")
    ax.set_xlabel('t')
    ax.set_ylabel(f"$\\langle \sigma_0^{df['s_0^?'][i]} (t) \\rangle$")
    plots.append(fig)

## Thermal values of observables

In [ ]:
%%time

bc = 'c'

graphs = []

for L in [8, 10, 12, 14]:

    job = dict(
        oper=models.tfim_z.H_dense,
        oper_params={
            'L' : L,
            'h' : hx,
            'hz': hz,
            'bc': bc,
        },
        solver=np.linalg.eigh,
        solver_params={},
    )
    evals, _ = data.jobs.obtain(**job)
    zzz = lambda x: np.exp(-x * evals)
    zz  = lambda x: sum(evals * zzz(x))
    z   = lambda x: sum(zzz(x))
    fig, ax = plt.subplots()
    ax.plot(np.arange(3, 10), [ zz(e) / z(e) for e in np.arange(3, 10) ])
    ax.set_title(f'$L={L}$')
    ax.set_xlabel('$ \\beta $')
    ax.set_ylabel('$ E_\\beta $')
    graphs.append(fig)

## Entanglement entropy growth with time